# CNN-RNN 结构用于图像处理

## 导入数据

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
from tensorflow import keras

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

  6938624/170498071 [>.............................] - ETA: 1:08:02

In [ ]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)